In [4]:
#Let's start by importing our csv's as DataFrames
import pandas as pd

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

print('Training: ', df_train.shape)
print('Test: ', df_test.shape)

Training:  (404290, 6)
Test:  (2345796, 3)


In [12]:
#Print info about our data frame
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [14]:
#It's a good idea to take only a small sample of the data
#We can remove this block when we are ready to test the whole dataset
sample_size = 10000
true_questions = df_train[df_train.is_duplicate == 1]
false_questions = df_train[df_train.is_duplicate == 0]

df_train = pd.concat([true_questions.sample(sample_size), false_questions.sample(sample_size)])
df_train = df_train.sample(df_train.shape[0])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 292283 to 294024
Data columns (total 6 columns):
id              20000 non-null int64
qid1            20000 non-null int64
qid2            20000 non-null int64
question1       20000 non-null object
question2       20000 non-null object
is_duplicate    20000 non-null int64
dtypes: int64(4), object(2)
memory usage: 1.1+ MB


In [21]:
#Firstly, we should clean the data. This will give our model the best chance to identify duplicates accurately. 
#This means removing ambiguous stop words.

import nltk
nltk.download("stopwords")
from tqdm import tqdm #for progressbar

def extractStopWords(question):
    stops = set(stopwords.words("english"))
    words = question.split()
    meaningful_words = [w.lower() for w in words if not w in stops]
    return meaningful_words      
        
tqdm.pandas(mininterval=2,ncols=60, desc='Question1 Progress')
df_train["cleaned_q1"] = df_train.question1.progress_apply(extractStopWords)

tqdm.pandas(mininterval=2,ncols=60, desc='Question2 Progress')
df_train["cleaned_q2"] = df_train.question2.progress_apply(extractStopWords)



Question1 Progress:   0%| | 1/20000 [02:05<698:57:19, 125.82s/it]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\G4M3M4ST3R\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.




Question1 Progress:   0%|         | 0/20000 [00:00<?, ?it/s]

Question1 Progress:  34%|▎| 6732/20000 [00:02<00:03, 3365.18it/s]

Question1 Progress:  67%|▋| 13413/20000 [00:04<00:01, 3357.70it/s]

Question1 Progress: 100%|█| 20000/20000 [00:05<00:00, 3356.81it/s]

Question2 Progress:   0%|         | 0/20000 [00:00<?, ?it/s]

Question2 Progress:  33%|▎| 6667/20000 [00:02<00:04, 3332.69it/s]

Question2 Progress:  66%|▋| 13297/20000 [00:04<00:02, 3327.13it/s]

Question2 Progress:  99%|▉| 19806/20000 [00:06<00:00, 3304.76it/s]

Question2 Progress: 100%|█| 20000/20000 [00:06<00:00, 3296.82it/s]

In [24]:
#Now we can finally begin constructing our CNN
#Let's first create word vectors for our questions
import gensim

questions = list(df_train['cleaned_q1']) + list(df_train['cleaned_q2'])

# tokenize
#i = 0
#for question in tqdm(questions):
#    questions[i] = list(gensim.utils.tokenize(question, deacc=True, lower=True))
#    i += 1
    
    

# Set values for various parameters
num_features = 300                   
num_workers = 16                    
iteration = 10                                                                                  
num_neg = 20 

print ("Training model...")
model = gensim.models.Word2Vec(questions, 
                          workers=num_workers,
                          size=num_features, 
                          iter=iteration,
            negative=num_neg)
print ("Training Complete!")

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# creta a dict 
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
print ("Number of tokens in Word2Vec:", len(w2v.keys()))

Training model...
Training Complete!
Number of tokens in Word2Vec: 6668


In [25]:
model.most_similar("sugar")

[('60', 0.986575722694397),
 ('food,', 0.9854674339294434),
 ('ages', 0.9844881892204285),
 ('weak', 0.983527660369873),
 ('worried', 0.9834195971488953),
 ('overweight', 0.9833500385284424),
 ('hand?', 0.9832106828689575),
 ('emma', 0.9830905795097351),
 ('persons', 0.9826271533966064),
 ('walk', 0.9822831749916077)]